In [1]:
# import needed library
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import math
import numpy as np
import random

# train test split 
from sklearn.model_selection import train_test_split

# Kfold
from sklearn.model_selection import KFold

# regex
import re
 

In [2]:
#load iris dataset
data_iris = load_iris()
iris_X, iris_y = load_iris(return_X_y=True)
feature_iris = data_iris['feature_names']

In [3]:
#transform iris into dataframe
iris_X=pd.DataFrame(iris_X)
iris_y=pd.DataFrame(iris_y)

In [4]:
#create index so be merge
iris_X=iris_X.reset_index()
iris_y=iris_y.reset_index()

In [5]:
iris_X.rename(columns = {0:feature_iris[0],1:feature_iris[1],2:feature_iris[2],3:feature_iris[3]}, inplace = True)

In [6]:
iris_y.rename(columns = {0:'target'}, inplace = True) 

In [7]:
#merge dataset iris
iris=iris_X.merge(iris_y)

In [8]:
#drop index
iris.drop("index",axis=1,inplace=True)

In [9]:
def entropy(parsed_data, target_attribute):
    parsed_value_target = {}
    total_value_target = 0
  
    for i in parsed_data[target_attribute]:
        if i is not None:
            if i not in parsed_value_target:
                parsed_value_target[i] = 1
            else:
                parsed_value_target[i] += 1

            total_value_target += 1
  
    log_result = 0

    for i in parsed_value_target:
        log_result += float(parsed_value_target[i])/total_value_target * math.log((float(parsed_value_target[i])/total_value_target), 2)
  
    return -1 * log_result

In [10]:
# hasn't handle after universal entropy
def information_gain(data, gain_attribute, target_attribute):
    gain_result = 0
    attribute_entropy_result = 0
    parsed_attribute_count = {}
    total_attribute_count = 0
    
    for i in data[gain_attribute]:
        if i is not None:
            if i not in parsed_attribute_count:
                parsed_attribute_count[i] = 1
            else:
                parsed_attribute_count[i] += 1
            
            total_attribute_count += 1
    
    for i in parsed_attribute_count:
        parsed_data = data.loc[data[gain_attribute]==i]
        attribute_entropy_result += float(parsed_attribute_count[i])/total_attribute_count * entropy(parsed_data, target_attribute)    

    gain_result += entropy(data,target_attribute) + (-1 * attribute_entropy_result)
    return gain_result

In [11]:
def split_information(data, gain_attribute):
    res = entropy(data, gain_attribute)
    if(res==0):
        res=0.00000001
    return res

In [12]:
def gain_ratio(data, gain_attribute, target_attribute):
    res = information_gain(data, gain_attribute, target_attribute) / split_information(data, gain_attribute)
    return res

In [13]:
def gain_ratio_continous(data, gain_attribute, target_attribute):
    res = information_gain_continous(data, gain_attribute, target_attribute)[0] / split_information(data, gain_attribute)
    return res,information_gain_continous(data, gain_attribute, target_attribute)[1]

In [14]:
def best_attribute(data,target_attribute,is_IG):
    gain_attribute = {
        'value': 0,
        'name': ''
    }
    
    
    for i in data.columns:
        if (i != target_attribute):
            if is_IG:
                if information_gain(data, i, target_attribute) > gain_attribute['value']:
                    gain_attribute['value'] = information_gain(data, i, target_attribute)
                    gain_attribute['name'] = i
            else:
                if gain_ratio(data, i, target_attribute) > gain_attribute['value']:
                    gain_attribute['value'] = gain_ratio(data, i, target_attribute)
                    gain_attribute['name'] = i
                

    return gain_attribute['name']

In [15]:
import math

class Node:
    def __init__(self, attribute=None, label=None, vertex=None):
        self.attribute = attribute
        self.label = label
        self.vertex = vertex
        self.children = {}
        self.most_common_label = None
    
    def set_most_common_label(self, most_common_label):
        self.most_common_label = most_common_label
        
    def get_most_common_label(self):
        return self.most_common_label
        
    def setAttribute(self, attribute):
        self.attribute = attribute

    def setLabel(self, label):
        self.label = label
        
    def setVertex(self, vertex):
        self.vertex = vertex
  
    def addChildren(self, attributeValue, node):
        self.children[attributeValue] = node
    
    def getChildren(self):
        return self.children
    
    def getLabel(self):
        return self.label
    
    def getVertex(self):
        return self.vertex

In [16]:
def get_most_common_label(data, target_attribute):
    parsed_value_target = {}
  
    for i in data[target_attribute]:
        if i is not None:
            if i not in parsed_value_target:
                parsed_value_target[i] = 1
            else:
                parsed_value_target[i] += 1

    most_common = {
        'value': 0,
        'name': ''
    }
    
    for i in parsed_value_target:
        if parsed_value_target[i] > most_common['value']:
            most_common['value'] = parsed_value_target[i]
            most_common['name'] = i
    
    return most_common['name']

In [17]:
def most_common_label_target(data,target_attribute):
    most_comm = None
    occ = 0;
    for i in data[target_attribute].unique():
        if data[data[target_attribute] == i].shape[0] >  occ:
            most_comm = i
            occ = data[data[target_attribute] == i].shape[0]
    return most_comm

In [18]:
def id3(data, target_attribute, is_IG):
    node = Node()
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
    
    elif len(data.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_ = best_attribute(data,target_attribute,is_IG)
        node.setAttribute(best_attribute_)
        for i in data[best_attribute_].unique():
            node.addChildren(i,id3(data.loc[data[best_attribute_]==i],target_attribute,is_IG))
            node.set_most_common_label(most_common_label_target(data,target_attribute))        
            
    return node

In [19]:
def print_tree(node,depth):
    if node.label is not None: 
        print("    "*(depth+1) +str(node.label))
    else:
        print("    "*depth + "["+ node.attribute +"]")
        for i in node.children:
            print("----"*(depth+1) +str(i))
            print_tree(node.children[i],depth+1)        

In [20]:
def copy_tree(node):
    temp_node = Node()
    if node.label is not None: 
        temp_node.setLabel(node.label)
    else:
        temp_node.setAttribute(node.attribute)
        for i in node.children:
            temp= Node()
            temp= node.children[i]
            temp_node.addChildren(i, temp)
    return temp_node

In [21]:
def check_tree(node,data,index,result, target_attribute):
    if node.label is not None: 
        result.append(node.getLabel())
    else:
        if data.loc[index, node.attribute] is None:
            result.append(node.get_most_common_label())
        for i in node.children:
            if i==data.loc[index,node.attribute]:
                check_tree(node.children[i],data,index,result, target_attribute)

In [22]:
def pred(data,model,target_attribute):
    result = []
    for i in range(len(data)):
        check_tree(model,data[i:i+1],i,result, target_attribute)
        data = {target_attribute:result}
    return pd.DataFrame(data)
    

In [23]:
# hasn't handle after universal entropy
def information_gain_continous(data, gain_attribute, target_attribute):
    gain_result = 0
    save_boundary = -1
    min_entropy = 999999 
    data = data.sort_values(gain_attribute)
    data=data.reset_index().drop('index',axis=1)
    length_data = len(data)
    for i in range(length_data):
        if (i!=length_data-1):
            if (data.loc[i,target_attribute] != data.loc[i+1,target_attribute]):
                temp_boundary = (data.loc[i,gain_attribute] + data.loc[i+1,gain_attribute])/2
                parsed_data_upper = data.loc[data[gain_attribute]>=temp_boundary]
                parsed_data_lower = data.loc[data[gain_attribute]<temp_boundary]
                len_parsed = len(parsed_data_upper)               
                temp_entropy = float(len_parsed)/length_data * entropy(parsed_data_upper, target_attribute) + float((length_data-len_parsed))/length_data * entropy(parsed_data_lower, target_attribute)     
                if temp_entropy < min_entropy:
                    #print(gain_attribute, temp_boundary, temp_entropy)
                    min_entropy = temp_entropy 
                    save_boundary = temp_boundary

    gain_result += entropy(data,target_attribute) + (-1 * min_entropy)
    return gain_result,save_boundary

In [24]:
information_gain_continous(iris, 'petal length (cm)', 'target')

(0.9182958340544894, 2.45)

In [25]:
def best_attribute_c45(data,target_attribute,is_IG):
    gain_attribute = {
        'value': 0,
        'name': '',
        'boundary': -99999999
    }
    
    
    for i in data.columns:
        if (i != target_attribute):
            if is_IG:
                if data[i].dtypes in [pd.np.dtype('float64'), pd.np.dtype('float32')]:
                    ig = information_gain_continous(data, i, target_attribute) 
                    if ig[0] > gain_attribute['value']:
                            gain_attribute['value'] = ig[0]
                            gain_attribute['name'] = i
                            gain_attribute['boundary'] = ig[1]
                else:
                    if information_gain(data, i, target_attribute) > gain_attribute['value']:
                            gain_attribute['value'] = information_gain(data, i, target_attribute)
                            gain_attribute['name'] = i
            else:
                if data[i].dtypes in [pd.np.dtype('float64'), pd.np.dtype('float32')]:
                    ig = gain_ratio_continous(data, i, target_attribute) 
                    if ig[0] > gain_attribute['value']:
                            gain_attribute['value'] = ig[0]
                            gain_attribute['name'] = i
                            gain_attribute['boundary'] = ig[1]
                else:
                    if gain_ratio(data, i, target_attribute) > gain_attribute['value']:
                            gain_attribute['value'] = gain_ratio(data, i, target_attribute)
                            gain_attribute['name'] = i

    return gain_attribute['name'],round(gain_attribute['boundary'],2)

In [26]:
best_attribute_c45(iris,'target',False)

('petal width (cm)', 0.8)

In [27]:
def c45(data, target_attribute,is_IG):
    node = Node()
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
    
    elif len(data.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_,bound  = best_attribute_c45(data,target_attribute,is_IG)
        #print(best_attribute_)
        node.setAttribute(best_attribute_)
        node.set_most_common_label(most_common_label_target(data,target_attribute))
        if bound==-99999999:
            for i in data[best_attribute_].unique():
                node.addChildren(i,c45(data.loc[data[best_attribute_]==i],target_attribute,is_IG))
        else:
            node.addChildren('>='+str(bound),c45(data.loc[data[best_attribute_]>=bound],target_attribute,is_IG))
            node.addChildren('<'+str(bound),c45(data.loc[data[best_attribute_]<bound],target_attribute,is_IG))
    return node

In [28]:
def c45_prun(data, target_attribute,is_IG,vertex_=None):
    node = Node(vertex = vertex_)
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
     
    elif len(data.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_,bound  = best_attribute_c45(data,target_attribute,is_IG)
        #print(best_attribute_)
        node.setAttribute(best_attribute_)
        node.set_most_common_label(most_common_label_target(data,target_attribute))
        if bound==-99999999:
            for i in data[best_attribute_].unique():
                node.addChildren(i,c45_prun(data.loc[data[best_attribute_]==i],target_attribute,is_IG,i))
        else:
            node.addChildren('>='+str(bound),c45_prun(data.loc[data[best_attribute_]>=bound],target_attribute,is_IG,'>='+str(bound)))
            node.addChildren('<'+str(bound),c45_prun(data.loc[data[best_attribute_]<bound],target_attribute,is_IG,'<'+str(bound)))
    return node

In [29]:
c45(iris, "target",True)

In [30]:
print_tree(c45(iris, "target",True),0)

[petal length (cm)]
---->=2.45
    [petal width (cm)]
-------->=1.75
        [sepal width (cm)]
------------>=3.15
            [sepal length (cm)]
---------------->=6.05
                    2
----------------<6.05
                    1
------------<3.15
                2
--------<1.75
        [petal length (cm)]
------------>=4.95
            [petal width (cm)]
---------------->=1.55
                [sepal length (cm)]
-------------------->=6.95
                        2
--------------------<6.95
                        1
----------------<1.55
                    2
------------<4.95
            [petal width (cm)]
---------------->=1.65
                    2
----------------<1.65
                    1
----<2.45
        0


In [31]:
def check_tree_c45(node,data,index,result, target_attribute):
    if node.label is not None: 
        result.append(node.getLabel())
    else:
        if data.loc[index, node.attribute] is None:
            result.append(node.get_most_common_label())
        else:
            for i in node.children:
                #print(i)
                if i[0]=='<':
                    #print(i,2)
                    bound=float(i[1:])
                    if bound>data.loc[index,node.attribute]:
                        #print(data.loc[index,node.attribute])
                        check_tree_c45(node.children[i],data,index,result, target_attribute)
                elif i[0]=='>':
                    bound=float(i[2:])
                    #print(i,1)
                    if bound<=data.loc[index,node.attribute]:
                        #print(data.loc[index,node.attribute])
                        check_tree_c45(node.children[i],data,index,result, target_attribute)
                else:
                    if i==data.loc[index,node.attribute]:
                        check_tree(node.children[i],data,index,result, target_attribute)
                    

In [32]:
def pred_c45(data,model,target_attribute):
    result = []
    for i in range(len(data)):
        check_tree_c45(model,data[i:i+1],i,result, target_attribute)
    data = { target_attribute: data[target_attribute]
            ,'prediction':result}
    return pd.DataFrame(data)
    

In [33]:
p=iris

In [34]:
p=p.drop('target',axis=1)

In [35]:
p=p.reset_index().drop('index',axis=1)

In [36]:
p['sepal width (cm)'] = None

In [37]:
def accuracy(pred,data):
    cnt = 0
    for i in range(len(pred)):
        if pred.loc[i] == data.loc[i]:
            cnt+=1
    return cnt*100/len(pred)

In [38]:
def get_data_validate(data):
    data_column = data.columns
    data_10 = pd.DataFrame(columns=data_column)
    data_90 = pd.DataFrame(columns=data_column)
    #print(data_column)
    count_10 = 0
    count_90 = 0
    
    for i in range(data.shape[0]):
        if(i%10 == 1):
            # Pass the row elements as key value pairs to append() function 
            data_10 = data_10.append(data.loc[[i]] , ignore_index=True)
        else:
            data_90 = data_90.append(data.loc[[i]] , ignore_index=True)
    return data_10, data_90

In [39]:
def prune_tree(node, attribute, vertex):
    if(node.label is not None):
        return node,0
    elif(node.attribute == attribute and node.vertex == vertex):
        node.children = {}
        node.attribute = None
        node.label = node.most_common_label
        
        return node,1
    else:
        for i in node.children:
            a,b = prune_tree(node.children[i], attribute, vertex)
            
        return node,b

In [40]:
def post_pruning(data, node, current_node, children_node, target_attribute):
    if children_node == 0:
        return node
    else:
        for i in current_node.children:
            if current_node.vertex is None: 
                #print(i,1)
                post_pruning(data, node, current_node.children[i], children_node-1, target_attribute)
            else:
                if current_node.attribute is not None:
                    data10, data90 = get_data_validate(data)
                    #print(current_node.attribute,2)
                    save = copy_tree(current_node)
                    temp_node,c = prune_tree(node, current_node.attribute, current_node.vertex)
                    train=data90.drop(target_attribute,axis=1)
                    #print_tree(node,0)
                    temp_node_pred = pred_c45(train, node, target_attribute)
                    #print(temp_node_pred)
                    temp_node_accuracy = accuracy(temp_node_pred[target_attribute], data90[target_attribute])
                    #print(temp_node_accuracy)
                    if temp_node_accuracy == 100:
                        post_pruning(data, temp_node, current_node, children_node, target_attribute)
                    else :
                        
                        node.addChildren(current_node.vertex, save)
                        #print_tree(node,0)


                        post_pruning(data, node, save.children[i], children_node, target_attribute)
        return node

### ANN

In [41]:
def sigmoid(x):
    return 1/(1+math.exp(-x))

In [42]:
def sign(x):
    if (x>0.5):
        return 1
    else:
        return 0

In [43]:
def count_error(target, output):
    return 0.5*(target-output)*(target-output)

In [44]:
class Neuron:
    def __init__(self, out=None, w=None, is_used=None, error=None):
        self.out = out
        self.w = []
        self.is_used = is_used
        self.error = 0
        self.deltaW = []
        
    def set_out(self, out):
        self.out = out
        
    def get_out(self):
        return self.out
    
    def add_deltaW(self, value):
        self.deltaW.append(value)
        
    def get_deltaW(self, index):
        return self.deltaW[index]
    
    def get_arrdW(self):
        return self.deltaW
    
    def set_deltaW(self, index, value):
        self.deltaW[index] = value
        
    def add_w(self, value):
        self.w.append(value)
        
    def set_w(self, index, value):
        self.w[index] = value
    
    def get_w(self, index):
        return self.w[index]
    
    def get_arrW(self):
        return self.w
    
    def set_is_used(self, is_used):
        self.is_used = is_used
        
    def get_is_used(self):
        return self.is_used
    
    def set_error(self, error):
        self.error = error
    
    def get_error(self):
        return self.error

In [45]:
x = [Neuron() for i in range (5)]
y = [x for i in range(5)]

In [46]:
def LenCol(arr):
    return len(arr)
def LenRow(inp,hid,out):
    return max(inp,max(hid)+1,out)
def MakeMatrix(row,col):
    return [[Neuron() for i in range (col)] for j in range(row)]

In [47]:
def printMatrixMLP(matrix):
    for i in range(len(matrix)):
        temp_matrix_out = []
        for j in range(len(matrix[i])):
            temp_matrix_out.append(matrix[i][j].get_arrW())
        print(temp_matrix_out)

In [48]:
def convTarget(output,unique):
    res = []
    for i in range(unique):
        if (i!=output):
            res.append(0)
        else:
            res.append(1)
    return res
    

In [49]:
def resetDeltaWeight(mat,layer):
    for i in range(len(layer)-1):
        for j in range(layer[i]):
            for k in range(layer[i+1]):
                mat[j][i].set_deltaW(k,0)

In [50]:
def initW(mat,layer):
    for i in range(len(layer)-1):
        for j in range(layer[i]):
            for k in range(layer[i+1]):
                mat[j][i].add_w(np.random.uniform(-1,1)) 
                mat[j][i].add_deltaW(0)

In [51]:
def feedforward(mat,layer):
    for i in range(1,len(layer)):
        if (i!=len(layer)-1):
            mat[layer[i]][i].set_out(1)
        for j in range(layer[i]):
            net = 0;
            for k in range(layer[i-1]):
                #print(k,i-1,j)
                net = net + mat[k][i-1].get_out()*mat[k][i-1].get_w(j)
#             if (i!=len(layer)-1):
            mat[j][i].set_out(sigmoid(net))
#             else:
#                 mat[j][i].set_out(sign(sigmoid(net)))

In [52]:
def get_error_total(matrix, n_output, target_arr):
    output_col = len(matrix[0])-1
    total_error = 0
    for i in range(n_output):
        total_error += 0.5*(target_arr[i] - matrix[i][output_col].get_out())**2
    
    return total_error

In [53]:
def set_output_error(matrix, row, col, target_value):
    output_value = matrix[row][col].get_out()
    #print(output_value, target_value)
#     print(output_value * (1 - output_value) * (target_value - output_value))
    matrix[row][col].set_error(output_value * (1 - output_value) * (target_value - output_value))

In [54]:
def set_hidden_error(matrix, row, col):
    output_value = matrix[row][col].get_out()
    delta_weight_error = 0
    for i in range(len(matrix[row][col].w)):
        delta_weight_error += matrix[row][col].get_w(i) * matrix[i][col+1].get_error()
    matrix[row][col].set_error(output_value * (1 - output_value) * delta_weight_error)

In [55]:
def update_delta_weight(matrix, row, col, idx , learning_rate):
    #print(matrix[i][col+1].get_error(),matrix[row][col].get_out(),row,col)
    #print(row,col+1)
    delta_weight = learning_rate * matrix[row][col].get_error() * matrix[idx][col-1].get_out()
    #print(delta_weight)
    
    current_weight = matrix[idx][col-1].get_deltaW(row)
    matrix[idx][col-1].set_deltaW(row, current_weight + delta_weight)
    #print(idx,col-1,row,current_weight + delta_weight)

In [56]:
def update_weight(mat,layer):
    for i in range(len(layer)-1):
        for j in range(layer[i]):
            for k in range(layer[i+1]):
                curr_weight = mat[j][i].get_w(k)
                delta_weight = mat[j][i].get_deltaW(k)
                mat[j][i].set_w(k,curr_weight + delta_weight) 

In [57]:
def backpropagation(matrix, layer, learning_rate, target_arr):
    for i in range(len(layer)-1, 0, -1):
        if i == len(layer)-1:
            for j in range(layer[i]):
                set_output_error(matrix, j, i, target_arr[j])
        else:
            for j in range(layer[i]):
                set_hidden_error(matrix, j, i)
        for j in range(layer[i-1]):
            for k in range(layer[i]):
                update_delta_weight(matrix, k, i, j, learning_rate)

In [58]:
hidden_node = [3,4]
def MLP(data,target, hidden_node, epochs, learning_rate):
    output_node = data[target].nunique()
    input_node = len(data.columns)
    hidden_node.insert(0,input_node)
    hidden_node.append(output_node)
    layer_node = hidden_node
    mlp = MakeMatrix(LenRow(input_node,hidden_node,output_node),LenCol(hidden_node))
    initW(mlp,layer_node)
    train = data.drop(target,axis=1)
    batch = len(data)/10;
    for k in range(epochs):
        error = 0
        for i in range(int(batch)):
            for j in range(10):
                for index,col in enumerate(train.columns):
                    mlp[index][0].set_out(data[col][j+i*10])
                mlp[input_node-1][0].set_out(1)
                feedforward(mlp,layer_node)
                backpropagation(mlp, layer_node, learning_rate, convTarget(data[target][j+i*10],output_node))
                error += get_error_total(mlp, len(convTarget(data[target][j+i*10],output_node)), convTarget(data[target][j+i*10],output_node))
            update_weight(mlp,layer_node)
            resetDeltaWeight(mlp,layer_node)
        print(error)
    print()
    print()
    printMatrixMLP(mlp)

    return mlp,layer_node

In [59]:
def predMyMLP(data, target, model, output_layer):
    out = []
    mlp = model[0]
    for i in range(0,data.shape[0],1):
        for index,col in enumerate(data.columns):
            mlp[index][0].set_out(data[col][i])
        mlp[len(data.columns)-1][0].set_out(1)
        feedforward(mlp,model[1])
        output_col = len(mlp[0])-1
        #print(output_col)
        mx=0
        res=-1
        for j in range(output_layer):
            # print(mlp[j][output_col].get_out())
            if mlp[j][output_col].get_out()>mx:
                mx = mlp[j][output_col].get_out()
                res = j
        # print()
        out.append(res)
    data = { target: data[target],
             'prediction':out}
    return pd.DataFrame(data)


# Testing

In [60]:
class_list = iris['target'].unique()

# Create confusion matrix from testing result of a model
#
# Input:
#    pandas.DataFrame   prediction_data
#    String             actual_column
#    String             prediction_column
#    Array              class_list
# Output:
#    pandas.DataFrame   confusion_matrix
#
def confusion_matrix(prediction_data, actual_column, prediction_column, class_list):
    actual_data = pd.Categorical(prediction_data[actual_column], categories=class_list)
    prediction_data = pd.Categorical(prediction_data[prediction_column], categories=class_list)
    confusion_matrix = pd.crosstab(actual_data, prediction_data, rownames=['Actual'], colnames=['Predicted'], dropna=False)
    return confusion_matrix

In [61]:
# Get accuracy from testing result of a model
#
# Input:
#    pandas.DataFrame   prediction_data
#    String             actual_column
#    String             prediction_column
# Output:
#    float              accuracy
#
def testing_accuracy(prediction_data, actual_column, prediction_column):
    count = 0
    for index, row in prediction_data.iterrows():
        if(row[actual_column] == row[prediction_column]):
            count = count + 1
    return round(count/prediction_data.shape[0],2)
            


## Train Test Split

In [62]:
# Drop index columns
iris_X.drop('index',axis=1,inplace=True);
iris_y.drop('index',axis=1,inplace=True);

In [63]:
# spliting data
X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size=0.1, random_state=42)

In [64]:
# merge data from train test split
# output is train and testing data
def mergeData(X_train, X_test, y_train, y_test):
    X_train = X_train.reset_index()
    y_train = y_train.reset_index()
    X_test = X_test.reset_index()
    y_test = y_test.reset_index()
    trainData = X_train.merge(y_train)
    testData = X_test.merge(y_test)
    trainData.drop("index",axis=1,inplace=True)
    testData.drop("index",axis=1,inplace=True)
    return trainData,testData

In [65]:
trainData,testData = mergeData(X_train, X_test, y_train, y_test)

## DTL

In [66]:
# training model
model = c45(trainData, "target",True)
# predict test
prediction_dtl = pred_c45(testData, model, "target")
print(prediction_dtl)

    target  prediction
0        1           1
1        0           0
2        2           2
3        1           1
4        1           1
5        0           0
6        1           1
7        2           2
8        1           1
9        1           1
10       2           2
11       0           0
12       0           0
13       0           0
14       0           0


In [67]:
print(confusion_matrix(prediction_dtl,'target','prediction', class_list))

Predicted  0  1  2
Actual            
0          6  0  0
1          0  6  0
2          0  0  3


In [68]:
testing_accuracy(prediction_dtl,'target','prediction')

1.0

## ANN

In [69]:
# training model
hidden_node = [4,4,2]
x = MLP(trainData, 'target', hidden_node,1500,0.1)

52.515267559816884
47.540938285991025
45.27896045714697
44.23186810250798
43.772437830007824
43.57958992726782
43.500078017731475
43.46713174667251
43.45309581062769
43.446749290088846
43.44356103566873
43.4416957217599
43.440404340302365
43.43937763514545
43.43848559799687
43.437671919085005
43.43691093055893
43.436189911719026
43.43550175678311
43.43484190541904
43.43420703861323
43.43359451078296
43.433002091614256
43.43242783992114
43.43187003510296
43.43132713482226
43.43079774551835
43.43028059991228
43.42977453882814
43.42927849600252
43.42879148513705
43.42831258871736
43.427840948254634
43.42737575568313
43.42691624569813
43.42646168885409
43.42601138527188
43.42556465882597
43.425120851699674
43.42467931920941
43.42423942480923
43.42380053519322
43.42336201541769
43.422923223966336
43.422483507681534
43.42204219648149
43.42159859777807
43.42115199050363
43.42070161864323
43.420246684157654
43.41978633916523
43.41931967723058
43.4188457235835
43.41836342406003
43.4178716325206

13.307942133312267
13.302519067098192
13.297117090080196
13.291743388027315
13.286404722695224
13.281107368288216
13.275857033080726
13.270658768393233
13.265516867944967
13.260434761483339
13.255414907469062
13.250458690407998
13.245566329081587
13.240736802344165
13.235967799233658
13.231255699794186
13.226595592168632
13.221981330150072
13.217405633494785
13.21286023096254
13.208336043398965
13.203823401408059
13.19931228953361
13.194792606657762
13.190254430800634
13.185688275881645
13.181085328395675
13.1764376533577
13.171738361134775
13.166981729660563
13.162163279688306
13.157279803839085
13.152329352937228
13.147311185267156
13.142225685813834
13.137074263242942
13.131859232408003
13.126583689667205
13.121251387411245
13.115866613102014
13.110434076938907
13.104958811114484
13.099446082567075
13.093901320229747
13.088330057029323
13.082737886304272
13.077130431870042
13.071513330642771
13.065892226506202
13.060272773944744
13.054660649834442
13.049061571655356
13.0434813202341

11.517514598334802
10.881343170804662
11.653581492100093
10.86876393130506
11.435693260847671
11.293580264556576
11.47129082466736
11.41889523607961
11.693065496415713
11.171670711016501
11.678252815958594
11.109238002017019
11.426005291326993
11.350763729953888
11.439968879383565
11.126534476493012
11.374577246435598
11.2662602864546
11.453335614858057
11.391128708167832
11.697032183906988
11.33659712985707
11.460984159599153
10.822736164747397
11.314920805652239
11.416031762307767
11.372320948553545
11.589931267062564
10.914266441730616
11.36824560400149
11.53506327618064
11.36340638548925
11.405775929274911
11.37341695201823
11.798714829198133
11.39950681019847
11.382936454076802
11.407186025432704
11.23460066280697
11.590535889919437
10.9969442936977
11.36145012048395
11.307669572821466
11.321832234607243
11.709466629665847
11.67851861517492
11.74946350100144
11.397197070241035
11.326496709350813
11.353404688216694
11.705488713293922
11.382206018770894
11.232607465938363
11.2517160

10.802429557965167
10.79635439767663
10.764385140597202
11.617805005912086
10.794010876556818
10.762905459352279
10.959116143214683
10.745972908304847
10.806250644486475
10.747073849053592
10.757089363000551
10.946917330362162
10.733919663104107
10.778114285443515
10.783535892791265
11.410794743444383
11.390336939784206
10.834357375747526
10.785875339774712
10.722660057115698
10.864601890527847
11.229117296142416
10.871999215052591
11.04424140131912
11.261905397796335
11.508163079182687
11.324756767235057
10.926589415492984
10.913545240500001
10.734023435109682
10.934499943494707
10.887276161806938
11.38388566992986
10.971704398655323
10.959396782663765
10.889039421245716
10.800906990862156
10.784000435750045
10.716612908805383
10.996509236572052
11.202927345492855
10.736667326650188
11.022900436472957
11.190759297337946
10.917417004686746
10.72059783427953
10.978518307421359
10.963723605633342
11.08886508594282
10.867282851045879
11.235773463353656
10.914865283645415
10.70910317632517

In [70]:
# predict
prediction_ann = predMyMLP(testData, 'target', x, 3)
print(prediction_ann)

    target  prediction
0        1           1
1        0           0
2        2           2
3        1           1
4        1           1
5        0           0
6        1           1
7        2           2
8        1           1
9        1           1
10       2           2
11       0           0
12       0           0
13       0           0
14       0           0


In [71]:
# confusion matrix
print(confusion_matrix(prediction_ann,'target','prediction', class_list))

Predicted  0  1  2
Actual            
0          6  0  0
1          0  6  0
2          0  0  3


In [72]:
# accurcy
testing_accuracy(prediction_ann,'target','prediction')

1.0

## Cross Validation

## DTL

In [73]:
# make kfold
kf = KFold(n_splits=10,shuffle=True)

In [74]:
# this function will create file to save our DTL model
# our model representation will be likely printed tree

def create_file_DTL(node, depth):
    
    if node.label is not None: 
        file.write(" "*(depth+1) +str(node.label))
        file.write('\n')
    else:
        file.write(" "*depth + "["+ node.attribute +"]")
        file.write('\n')
        for i in node.children:
            file.write("----"*(depth+1) +str(i))
            create_file_DTL(node.children[i],depth+1)        

In [75]:
# load file txt representation of DTL and return an model as root

def load_file_dtl(parent, depth, file):
    line = file.readline().rstrip()

    while line:
        tabs = line.count('----')

        if tabs < depth:
            break
        else :
            node = Node()
            parsed_line = ''
            vertex = ''

            if depth == 0:
                line = line[1:-1]
                parent.setAttribute(line)

            else:
                if (line[-1] == ']'):
                    parsed_line = line.replace('----', '').split('[')
                    vertex = parsed_line[0].rstrip()
                    attribute = parsed_line[1].strip()[:-1]
                    node.setAttribute(attribute)
                    node.setVertex(vertex)
                    
                else:
                    parsed_line = line.replace('----', '').split(' ' * (depth+1))
                    vertex = parsed_line[0].rstrip()
                    label = parsed_line[1].strip()
                    node.setVertex(vertex)
                    node.setLabel(label)

            parent.children[vertex] = node
            
            line = load_file_dtl(node, depth + 1, file)
    
    return line 

In [76]:
# save DTL model into file
file = open("text_dtl.txt", "w")
create_file_DTL(model, 0)
file.close()

In [77]:
# load DTL model from file
node = Node() # node is a parent node
file = open("text_dtl.txt", "r")

#call the function to load DTL model from file, 0 (depth) represent a root node
load_file_dtl(node, 0 , file)
node.children[''].setAttribute(node.attribute)
node = node.children['']
print_tree(node, 0)
file.close()

[petal length (cm)]
---->=2.45
    [petal width (cm)]
-------->=1.8
        [sepal length (cm)]
------------>=5.95
                2
------------<5.95
            [sepal width (cm)]
---------------->=3.1
                    1
----------------<3.1
                    2
--------<1.8
        [petal length (cm)]
------------>=4.95
            [petal width (cm)]
---------------->=1.55
                [sepal length (cm)]
-------------------->=6.95
                        2
--------------------<6.95
                        1
----------------<1.55
                    2
------------<4.95
            [petal width (cm)]
---------------->=1.65
                    2
----------------<1.65
                    1
----<2.45
        0


In [78]:
accuracy_DTL_KF = []
for train_index, test_index in kf.split(iris_X):
    X_train, X_test = iris_X.loc[train_index], iris_X.loc[test_index]
    y_train, y_test = iris_y.loc[train_index], iris_y.loc[test_index]
    trainData,testData =  mergeData(X_train, X_test, y_train, y_test)
    # training model
    model = c45(trainData, "target",True)
    # predict
    prediction_dtl = pred_c45(testData, model, "target")
    # save accuracy result from kfold
    accuracy_DTL_KF.append(testing_accuracy(prediction_dtl,'target','prediction'))

In [79]:
accuracy_DTL_KF

[0.93, 0.93, 1.0, 0.93, 0.93, 0.8, 0.93, 1.0, 0.93, 1.0]

In [80]:
round(np.mean(accuracy_DTL_KF)*100,2)

93.8

## ANN

In [81]:
# this function will create file to save our ANN model
# we save for each line represent a node j on layer i
# for every line we print into file in format; i,j = array_W | array_dW | out | error

def create_file_ANN():
    file = open("text_ann.txt", "w")

    for i in range(0,len(x[1])):
        for j in range(0,x[1][i]):
            file.write(str(i));
            file.write(',')
            file.write(str(j));
            file.write('=')
            file.write(str(x[0][i][j].get_arrW()))
            file.write('|')
            file.write(str(x[0][i][j].get_arrdW()))
            file.write('|')
            file.write(str(x[0][i][j].get_out()))
            file.write('|')
            file.write(str(x[0][i][j].get_error()))
            file.write('\n');

    file.close()

In [82]:
# load file txt that contain model representation of ANN

def load_file_ann(file):
    neurons = file.read().split("\n")[:-1]

    rows = []
    columns = []

    curr_row = 0

    for neuron in neurons:
        neuron_pos = neuron.split('=')[0].split(',')
        if int(neuron_pos[0]) != curr_row:
            rows.append(columns)
            columns = []
            curr_row += 1

        new_neuron = Neuron()
        neuron_elements = neuron.split('=')[1].rstrip().split('|')
        weights = neuron_elements[0]
        delta_weights = neuron_elements[1]
        out = neuron_elements[2]
        err = neuron_elements[3]

        if len(weights) > 2:
            parsed_weights = weights[1:-1].split(',')
            for weight in parsed_weights:
                new_neuron.add_w(float(weight.strip()))
        
        if len(delta_weights) > 2:
            parsed_delta_weights = delta_weights[1:-1].split(',')
            for delta_weight in parsed_delta_weights:
                new_neuron.add_deltaW(float(delta_weight.strip()))

        new_neuron.set_out(float(out))
        new_neuron.set_error(float(err))

        columns.append(new_neuron)

    max_col_len = max([len(row) for row in rows])
    for row in rows:
        if len(row) < max_col_len:
            for i in range(max_col_len-len(row)):
                row.append(Neuron())

    return rows

In [83]:
# save ANN model into file
create_file_ANN()

In [84]:
# load ANN model from file
file = open("text_ann.txt", "r")
rows = load_file_ann(file)
printMatrixMLP(rows)
file.close()

[[-1.0347032432056988, -0.9346717476106418, 5.075069310492819, -0.9217215062645336], [-0.17196609983664568, -0.361349123088827, 0.435394734830331, 0.1455436097340295], [-6.6428428501025705, 2.4512634865325955], [3.4557390616709345, -2.0204404413656167, -21.34225727612867], []]
[[-0.1500994627317474, 3.049618441832311, 8.78732237598905, -0.45684980572209416], [-4.941694676541726, 3.22338539149212, -2.752526476595043, -3.1820111366795194], [4.772819223570445, -4.939197586004817], [-20.889806467853056, -1.7823861020191827, 3.108726248065107], []]
[[0.203660695461699, -0.6263626713030249, -9.324456404702131, -0.32832440397764046], [2.3578172501676575, 3.8939949741058837, -0.9893660131443363, -5.337881143798171], [-4.136014971932633, 2.035018116954165], [], []]
[[-0.4112430219041036, -2.008327363064544, -11.126792774126525, -0.4887035292742222], [0.4982936176152737, 0.615118362349265, 0.2839728776701104, 0.894900153849292], [], [], []]


In [85]:
accuracy_MLP_KF = []
for train_index, test_index in kf.split(iris_X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = iris_X.loc[train_index], iris_X.loc[test_index]
    y_train, y_test = iris_y.loc[train_index], iris_y.loc[test_index]
    trainData,testData =  mergeData(X_train, X_test, y_train, y_test)
    hidden_node = [4,4,2]
    # train
    model = MLP(trainData, 'target', hidden_node,1500,0.1)
    # predict
    prediction_ann = predMyMLP(testData, 'target', x, 3)
    # save accuracy result from kfold
    accuracy_MLP_KF.append(testing_accuracy(prediction_ann,'target','prediction'))

48.95799336430583
47.75744288794553
46.77044407454109
45.91923273168538
45.201447122303286
44.6351337806101
44.22464784566579
43.95062057411144
43.77955950343474
43.67757276194084
43.61803562601582
43.582916468272046
43.56108863522736
43.54609221798845
43.53433086103554
43.523863439817205
43.51366758336753
43.5032148192326
43.4922335973483
43.48057976822285
43.468166491297836
43.45492625954665
43.44078996895524
43.42567486354431
43.40947694390872
43.392065409629964
43.37327771872714
43.352914343986015
43.33073252880403
43.30643841330678
43.27967686516632
43.25001822143173
43.21694091825725
43.17980862728667
43.13783998205631
43.09006818624635
43.035286623613935
42.97197484367971
42.89819667993398
42.81145831532379
42.70850816932247
42.58505158803617
42.43534026649608
42.25157804657137
42.023061865341354
41.73495676815918
41.3666164056533
40.88949190836618
40.26512895630714
39.44489474968559
38.37511222287509
37.01287968407431
35.35417268914342
33.46342879788635
31.484282230797504
29.61

15.52645016707568
15.526143440402539
15.525836957485124
15.525530330307339
15.525223194249534
15.524915208729915
15.52460605776662
15.524295450455318
15.523983121357993
15.523668830798703
15.523352365062951
15.523033536497792
15.52271218351059
15.522388170465037
15.52206138747384
15.521731750088577
15.521399198887815
15.52106369896556
15.520725239323239
15.520383832168916
15.520039512128557
15.519692335374963
15.519342378680493
15.51898973840057
15.518634529395525
15.518276883898704
15.517916950339085
15.517554892127306
15.517190886413479
15.51682512282593
15.516457802199442
15.516089135301497
15.515719341564962
15.515348647835062
15.514977287138054
15.514605497478588
15.514233520672255
15.513861601218826
15.51348998522152
15.513118919356538
15.512748649896663
15.512379421791998
15.51201147781007
15.511645057737153
15.511280397641768
15.510917729200902
15.510557279088923
15.510199268428549
15.50984391230296
15.50949141932758
15.50914199127988
15.508795822785112
15.508453101055697
15.50

15.146035209705968
15.145022943244902
15.144011014407361
15.142999370742562
15.141987965716242
15.140976758486431
15.139965713675181
15.138954801137071
15.137943995726069
15.136933277061447
15.13592262929402
15.134912040873644
15.133901504318699
15.132891015988594
15.131880575859944
15.130870187307108
15.129859856887695
15.128849594133612
15.127839411348154
15.126829323409396
15.12581934758048
15.12480950332691
15.123799812140955
15.122790297373768
15.121780984074892
15.120771898839397
15.119763069662755
15.118754525803222
15.117746297652033
15.116738416610806
15.115730914976538
15.114723825833675
15.113717182953291
15.112711020699058
15.111705373939833
15.110700277968576
15.109695768427411
15.108691881238496
15.10768865254035
15.106686118629575
15.105684315907347
15.10468328083065
15.103683049867767
15.102683659457796
15.101685145973795
15.100687545689304
15.099690894747935
15.098695229135659
15.0977005846555
15.096706996904379
15.09571450125175
15.094723132819817
15.093732926464952
1

14.81517405468683
14.814627872805469
14.814081862593179
14.813536013831065
14.812990316313742
14.812444759850745
14.811899334267958
14.811354029409003
14.810808835136633
14.810263741334188
14.809718737906941
14.80917381478351
14.80862896191734
14.808084169287955
14.807539426902503
14.806994724797077
14.806450053038104
14.805905401723757
14.805360760985344
14.804816120988626
14.804271471935284
14.80372680406422
14.803182107652948
14.802637373018928
14.802092590520958
14.801547750560486
14.801002843582935
14.80045786007903
14.799912790586177
14.799367625689706
14.798822356024152
14.79827697227467
14.79773146517814
14.797185825524597
14.796640044158385
14.796094111979453
14.795548019944569
14.795001759068562
14.794455320425515
14.793908695149952
14.793361874438078
14.79281484954888
14.792267611805315
14.791720152595444
14.791172463373565
14.790624535661347
14.790076361048849
14.789527931195686
14.788979237832018
14.788430272759651
14.787881027853038
14.787331495060295
14.786781666404183
1

15.971653323843315
15.945828883490524
15.92029962763357
15.895059355835945
15.870101905768502
15.84542118342222
15.821011189334127
15.796866040968672
15.772979991441808
15.749347444807652
15.725962968152388
15.702821300756451
15.67991736059464
15.65724624844748
15.634803249894043
15.612583835450653
15.590583659109617
15.568798555519484
15.547224536033665
15.525857783838392
15.504694648353727
15.483731639084127
15.462965419077541
15.442392798134602
15.42201072589306
15.40181628489592
15.381806683736963
15.36197925036259
15.342331425595487
15.322860756933379
15.303564892664307
15.28444157632979
15.265488641557356
15.246704007275955
15.228085673319185
15.209631716415632
15.191340286558408
15.173209603741887
15.155237955048277
15.137423692063269
15.119765228596805
15.102261038682403
15.084909654826586
15.067709666478404
15.050659718688797
15.033758510928237
15.017004796032317
15.000397379244939
14.983935117330173
14.967616917725191
14.95144173770875
14.935408583562019
14.919516509700784
14

...

20.87481381841586
20.874633213494718
20.874452979885056
20.874273116271333
20.87409362134446
20.87391449380163
20.87373573234642
20.87355733568857
20.873379302544148
20.873201631635368
20.873024321690576
20.872847371444255
20.87267077963696
20.872494545015257
20.872318666331758
20.87214314234498
20.871967971819426
20.871793153525424
20.87161868623919
20.87144456874274
20.871270799823872
20.87109737827614
20.87092430289881
20.870751572496754
20.87057918588057
20.87040714186641
20.870235439276
20.87006407693662
20.869893053681018
20.869722368347432
20.869552019779537
20.869382006826385
20.86921232834241
20.86904298318738
20.868873970226364
20.868705288329714
20.868536936372983
20.868368913236967
20.86820121780761
20.868033848976026
20.867866805638418
20.867700086696072
20.867533691055343
20.867367617627558
20.867201865329115
20.867036433081296
20.866871319810354
20.866706524447405
20.866542045928494
20.86637788319444
20.866214035190914
20.866050500868376
20.86588727918202
20.865724369091

20.81861191733443
20.818518229801718
20.818424585581567
20.818330982836237
20.818237419678276
20.81814389416847
20.818050404313862
20.817956948065536
20.817863523316326
20.817770127898562
20.817676759581452
20.81758341606858
20.817490094995136
20.817396793925035
20.81730351034796
20.817210241676204
20.81711698524131
20.81702373829064
20.81693049798371
20.816837261388407
20.816744025476805
20.816650787121095
20.816557543088983
20.81646429003903
20.816371024515732
20.816277742944205
20.816184441624806
20.81609111672718
20.815997764284226
20.815904380185675
20.81581096017109
20.8157174998228
20.815623994558237
20.815530439621867
20.81543683007662
20.815343160794892
20.815249426448975
20.815155621500942
20.815061740191826
20.81496777653033
20.814873724280645
20.814779576949604
20.814685327773155
20.814590969701683
20.814496495384724
20.814401897154504
20.814307167008405
20.814212296590433
20.814117277171285
20.814022099627234
20.81392675441745
20.813831231559902
20.813735520605526
20.81363

16.311494868835297
16.269865730384304
16.229278332773426
16.18969200930743
16.15107386507273
16.113391057799053
16.07660588646418
16.040673743809027
16.005543353981892
15.97115841752801
15.937459827800986
15.904387866162626
15.87188406495481
15.839892644485348
15.808361554499694
15.777243198645674
15.746494922407981
15.716079326263756
15.685964442245497
15.656123791749593
15.626536328599357
15.597186264862675
15.56806277722525
15.539159597485208
15.510474499979884
15.482008709069373
15.453766258696682
15.425753341431392
15.3979776850414
15.370447990395908
15.34317345632472
15.316163406629666
15.289427023679684
15.26297318358898
15.236810380954973
15.210946726920673
15.185390002740014
15.160147751510843
15.13522739260132
15.110636345868837
15.086382155527787
15.062472606107544
15.038915825151904
15.015720369065498
14.992895289812626
14.970450181075
14.948395203063432
14.926741085566615
14.90549910913439
14.884681064668142
14.864299192251705
14.844366100890584
14.824894671948817
14.80589

13.227743007222141
13.223253913254064
13.218739001031734
13.214199860534656
13.209637872807134
13.205054257724672
13.200450109810511
13.195826425351543
13.191184123130606
13.186524060435914
13.181847045549612
13.177153847589206
13.17244520434346
13.167721828576171
13.162984413150534
13.158233635237877
13.153470159810242
13.148694642567282
13.143907732412984
13.139110073569725
13.134302307397745
13.12948507397119
13.124659013451119
13.119824767285499
13.114982979259784
13.110134296415337
13.10527936984906
13.10041885540377
13.09555341425665
13.090683713410337
13.085810426090262
13.08093423205038
13.07605581778846
13.071175876671795
13.066295108973538
13.06141422182011
13.05653392904941
13.051654950980533
13.046778014095246
13.041903850632146
13.03703319809473
13.0321667986754
13.027305398597104
13.022449747376132
13.017600597009135
13.012758701088469
13.007924813850625
13.00309968916323
12.998284079456175
12.99347873460354
12.98868440076322
12.983901819181737
12.979131724971932
12.97437

12.049428748016146
12.047018209263792
12.044596953916075
12.04216483079186
12.039721691461288
12.037267390803326
12.03480178760798
12.032324745224479
12.029836132257476
12.02733582331345
12.02482369979821
12.022299650767325
12.019763573830808
12.017215376111956
12.014654975261243
12.012082300525002
12.00949729386783
12.006899911147302
12.004290123338892
12.001667917807534
11.999033299621985
11.996386292906744
11.993726942224647
11.991055313983047
11.988371497853732
11.985675608196603
11.98296778547455
11.980248197646143
11.977517041520704
11.974774544059208
11.972020963602551
11.969256591007838
11.966481750671804
11.963696801419738
11.960902137237706
11.958098187825161
11.955285418945575
11.95246433255342
11.949635466676208
11.946799395032949
11.943956726372047
11.941108103514974
11.938254202095267
11.935395728986688
11.93253342041972
11.929668039789078
11.926800375162466
11.923931236504725
11.921061452638634
11.918191867968261
11.91532333899674
11.912456730675189
11.909592912623342
11

In [86]:
accuracy_MLP_KF

[0.93, 1.0, 0.93, 0.93, 1.0, 1.0, 1.0, 0.93, 0.93, 1.0]

In [87]:
round(np.mean(accuracy_MLP_KF)*100,2)

96.5

# Generate New Instance

In [88]:
# Generate n instance like iris dataset
# return new n instance as new dataframe
def generateIris(n):
    a = []
    b = []
    c = []
    d = []
    e = []
    for i in range(n):
        a.append(random.uniform(4.5, 7.5))
        b.append(random.uniform(2, 4.4))
        c.append(random.uniform(1, 6.9))
        d.append(random.uniform(0.1, 2.5))
        e.append(1)
    iris = {'sepal length (cm)': a,
            'sepal width (cm)': b,
            'petal length (cm)':c,
            'petal width (cm)':d,
            'target': e
            }
    return pd.DataFrame(iris, columns = ['sepal length (cm)', 'sepal width (cm)','petal length (cm)','petal width (cm)','target'])

In [89]:
# Make 100 instance
new_ins = generateIris(100)

In [90]:
# save model
model = c45(iris, "target",True)
file = open("text_dtl.txt", "w")
create_file_DTL(model, 0)
file.close()

In [91]:
# load model
node = Node()
file = open("text_dtl.txt", "r")
load_file_dtl(node, 0 , file)
node.children[''].setAttribute(node.attribute)
node = node.children['']
file.close()

In [92]:
# prediction model
prediction_dtl = pred_c45(new_ins, node, "target")

In [93]:
new_ins['target']=prediction_dtl['prediction']

In [94]:
new_ins['target'].value_counts()

1    40
2    39
0    21
Name: target, dtype: int64